# Automating Slurm Script Generation with Python

In this session, we'll explore how to streamline the process of generating Slurm scripts tailored to specific commands or tasks. Slurm serves as a robust workload manager utilized in high-performance computing (HPC) environments to efficiently schedule and manage computational tasks across clusters of machines.

As you delve into complex bioinformatic projects, you'll find yourself frequently interacting with the Slurm scheduler, often writing repetitive scripts to execute various jobs. By leveraging Python, we aim to automate the generation of Slurm scripts, ultimately enhancing your productivity and efficiency.

Note that there is an online SLURM script generator for Nova available at [https://www.hpc.iastate.edu](https://www.hpc.iastate.edu/guides/nova/slurm-script-generator-for-nova). However, this session will focus on creating a custom Python script to generate Slurm scripts directly on your terminal.

## Learning Objectives

- Understand the role of Slurm in HPC environments.
- Learn how to automate the generation of Slurm scripts using Python.
- Customize Slurm job parameters and resource allocations.
- Basic error handling and debugging techniques.



## Before we begin

### Update your course repository

You need to clone the course repository to Nova. You probably already have cloned this, so you can skip this step.

```bash
git clone git@github.com:EEOB-BioData/BCB546_Spring2024.git
```

You will still need to pull new changes to this repository at the beginning of class. This will enable you to access new data files and scripts needed for in-class activities.

```bash
cd BCB546_Spring2024
git pull
```

Note that if you have modified any files in the repository, you will need to commit those changes before you can pull new changes. If you don't care about the changes, just delete and re-clone the repository.

### Start Jupyter notebook on Nova on demand.

You can start Jupyter notebook on Nova on demand. This will allow you to run Jupyter notebook on the server and access it from your local machine.

1. Go to the [Nova OnDemand](https://nova-ondemand.its.iastate.edu/) and login
2. Under the "Interactive Apps" tab, click on "Jupyter Notebook", request desired resources and click "Launch"
3. Wait for the job to start and click on the "Connect to Jupyter" button


## Slurm Overview

Slurm (Simple Linux Utility for Resource Management) is an open-source, fault-tolerant, and highly scalable workload manager designed for Linux clusters. It provides a robust framework for job scheduling, resource management, and job monitoring across a cluster of machines. Slurm is widely used in high-performance computing (HPC) environments to efficiently manage computational tasks and optimize resource utilization.

On Nova, we use Slurm scheduler for managing the jobs on the cluster. You can submit jobs to the Slurm scheduler using the `sbatch` command. You ran a job on the cluster in the previous session and a typical job script looks like this:

|  Slurm script                                          |   Description                               |
|--------------------------------------------------------|---------------------------------------------|
| `#!/bin/bash`                                          |  shebang line                               |
| `#SBATCH --nodes=1`                                    |  number of nodes                            |
| `#SBATCH --ntasks-per-node=4`                          |  processor core(s) per node                 |
| `#SBATCH --partition=nova`                             |  partition name                             |
| `#SBATCH --mem=24GB`                                   |  memory per node                            |
| `#SBATCH --time=24:00:00`                              |  walltime limit (HH:MM:SS or   DD-HH:MM:SS) |
| `#SBATCH --account=mhufford-lab`                       |  account name                               |
| `#SBATCH --job-name=JOBNAME`                           |  job name                                   |
| `#SBATCH --output=sbatch_stdout.txt`                   |  output file                                |
| `#SBATCH --error=sbatch_stdout.txt`                    |  error file                                 |
| `#SBATCH   --mail-user=username@domain.com`            |  email address                              |
| `#SBATCH --mail-type=begin`                            |  email at the start of the job              |
| `#SBATCH --mail-type=end`                              |  email at the end of the job                |
| `#SBATCH --mail-type=fail`                             |  email on job failure                       |
| `module load modulename1`                              |  load the required module                   |
| `samtools view -bS --threads=4 file.sam   > file.bam`  |  command to run                             |



If you have a lot of jobs to run, you might find yourself writing a lot of these scripts. In this session, we will learn how to automate the generation of these scripts using Python.

You can also simplify the process by submitting array jobs, but most often, for heterogeneous jobs, you will need to write individual scripts.

We will write both a simple script and a more complex script that can be used to generate Slurm scripts for different types of jobs.


## Automating Slurm Script Generation

We will use python to write various scripts that can generate Slurm scripts for different types of jobs.

### Simple Interactive Script Generator

We will start by writing a simple Python script that generates a Slurm script for a given command. It will prompt for the options and once collected it will write a slurm job.


`input` function is used to get the input from the user. It takes a string as an argument and returns the user input as a string.


In [11]:
input("Enter the number of nodes: ")

Enter the number of nodes: 1


'1'

We can use this to collect all the required options for the Slurm script.

In [12]:
nodes = input("Enter the number of nodes: ")
ntasks_per_node = input("Enter the number of tasks per node: ")
partition = input("Enter the partition name: ")
mem = input("Enter the memory per node: ")
time = input("Enter the walltime limit (HH:MM:SS or DD-HH:MM:SS): ")
account = input("Enter the account name: ")
job_name = input("Enter the job name: ")
output = input("Enter the output file: ")
error = input("Enter the error file: ")
mail_user = input("Enter the email address: ")
mail_type = input("Enter the email type (begin, end, fail): ")
module = input("Enter the module name: ")
command = input("Enter the command to run: ")

Enter the number of nodes: 1
Enter the number of tasks per node: 4
Enter the partition name: nova
Enter the memory per node: 24GB
Enter the walltime limit (HH:MM:SS or DD-HH:MM:SS): 1:00:00
Enter the account name: mhufford-lab
Enter the job name: 
Enter the output file: stdout
Enter the error file: stderr
Enter the email address: username@domain.edu
Enter the email type (begin, end, fail): begin
Enter the module name: samtools
Enter the command to run: samtools index genome.fasta


And once collected, you can use the print function to output the script.

In [13]:
print(f"""#!/bin/bash
#SBATCH --nodes={nodes}
#SBATCH --ntasks-per-node={ntasks_per_node}
#SBATCH --partition={partition}
#SBATCH --mem={mem}
#SBATCH --time={time}
#SBATCH --account={account}
#SBATCH --job-name={job_name}
#SBATCH --output={output}
#SBATCH --error={error}
#SBATCH --mail-user={mail_user}
#SBATCH --mail-type={mail_type}
module load {module}
{command}
""")

#!/bin/bash
#SBATCH --nodes=1
#SBATCH --ntasks-per-node=4
#SBATCH --partition=nova
#SBATCH --mem=24GB
#SBATCH --time=1:00:00
#SBATCH --account=mhufford-lab
#SBATCH --job-name=
#SBATCH --output=stdout
#SBATCH --error=stderr
#SBATCH --mail-user=username@domain.edu
#SBATCH --mail-type=begin
module load samtools
samtools index genome.fasta



You can save this script as `simple_slurm_script.py` and run it using the following command.

```python
python simple_slurm_script_v1.py
```

and the resulting job script will be printed to the terminal. You can then copy and paste it to file to generate the job.

Few things that can make this better

1. providing default values for the options
2. error handling for the inputs (eg. checking if the input is a number)
3. writing the script to a file directly

For the first point, you can provide default values for the options by using the `or` operator.

```python
nodes = input("Enter the number of nodes: ") or "1"
```
So when the user just presses enter, it will default to 1.

For the second point, you can use a `while` loop to keep asking for the input until the user provides a valid input.

```python
while True:
    try:
        nodes = int(input("Enter the number of nodes: ") or "1")
        break
    except ValueError:
        print("Please enter a number")
```

For the third point, you can use the `open` function to write the script to a file.

```python
with open("job.sh", "w") as f:
    f.write(f"""#!/bin/bash
#SBATCH --nodes={nodes}
#SBATCH --ntasks-per-node={ntasks_per_node}
#SBATCH --partition={partition}
#SBATCH --mem={mem}
#SBATCH --time={time}
#SBATCH --account={account}
#SBATCH --job-name={job_name}
#SBATCH --output={output}
#SBATCH --error={error}
#SBATCH --mail-user={mail_user}
#SBATCH --mail-type={mail_type}
module load {module}
{command}
""")
```


Let's write a script that does all of this.


In [15]:
while True:
    try:
        nodes = int(input("Enter the number of nodes: ") or "1")
        ntasks_per_node = int(input("Enter the number of tasks per node: ") or "4")
        partition = input("Enter the partition name: ") or "nova"
        mem = input("Enter the memory per node: ") or "24GB"
        time = input("Enter the walltime limit (HH:MM:SS or DD-HH:MM:SS): ") or "24:00:00"
        account = input("Enter the account name: ") or "mhufford-lab"
        job_name = input("Enter the job name: ") or "my-cool-job"
        output = input("Enter the output file: ") or "sbatch_stdout.txt"
        error = input("Enter the error file: ") or "sbatch_stderr.txt"
        mail_user = input("Enter the email address: ") or "username@domain.com"
        mail_type = input("Enter the email type (begin, end, fail): ") or "begin"
        module = input("Enter the module name: ") or "samtools"
        command = input("Enter the command to run: ")
        break
    except ValueError:
        print("Please enter a valid positive integer for numeric inputs")

with open(input("Enter the filename for the Slurm script (e.g., job.sh): ") or 'job.sh', "w") as f:
    f.write(f"""#!/bin/bash
#SBATCH --nodes={nodes}
#SBATCH --ntasks-per-node={ntasks_per_node}
#SBATCH --partition={partition}
#SBATCH --mem={mem}
#SBATCH --time={time}
#SBATCH --account={account}
#SBATCH --job-name={job_name}
#SBATCH --output={output}
#SBATCH --error={error}
#SBATCH --mail-user={mail_user}
#SBATCH --mail-type={mail_type}

module load {module}
{command}
""")

Enter the number of nodes: 1
Enter the number of tasks per node: 4
Enter the partition name: nova
Enter the memory per node: 
Enter the walltime limit (HH:MM:SS or DD-HH:MM:SS): 
Enter the account name: 
Enter the job name: index
Enter the output file: index_job
Enter the error file: 
Enter the email address: 
Enter the email type (begin, end, fail): 
Enter the module name: 
Enter the command to run: samtools index genome.fasta
Enter the filename for the Slurm script (e.g., job.sh): 


You won't see any output, but you can check the `job.sh` file to see the generated script.


You can save this script as `simple_slurm_script.py` and run it using the following command.

```bash
python simple_slurm_script_v2.py
```

Now, what if we have multi-line command that we need to provide as input?


Option 1.  Keep asking for the input until the user provides an empty line.

```python
command = []
while True:
    line = input("Enter the command to run (press enter to finish): ")
    if not line:
        break
    command.append(line)
```

Option 2. Take input as a file and read the file.

```python
filename = input("Enter the filename containing the command to run: ")
with open(filename) as f:
    command = f.readlines()
```

You can also ask the user (you!) if you want to provide a file or type the command.

```python
command = input("Do you want to provide the command as a file? (y/n): ")
if command.lower() == "y":
    filename = input("Enter the filename containing the command to run: ")
    with
    open(filename) as f:
        command = f.readlines()
else:
    command = []
    while True:
        line = input("Enter the command to run (press enter to finish): ")
        if not line:
            break
        command.append(line)
```

I will leave it up to you to implement this in the script.

### Advanced Script Generator

We will now write a more advanced script that can generate Slurm scripts for different types of jobs. We will use the `argparse` module to parse the command-line arguments and generate the Slurm script. We will also implement error handling and write the script to a file directly.

`argparse` module provides a mechanism to parse the command-line arguments and generate help messages. It is a standard module in Python and you can use it to write user-friendly command-line interfaces.

You can start by importing the `argparse` module and creating a parser object.

Main parts of the script:

```python
import argparse # import the argparse module
parser = argparse.ArgumentParser(description="Generate Slurm scripts for different types of jobs") # create a parser object
```

You can add arguments to the parser object using the `add_argument` method.

```python
parser.add_argument("--nodes", type=int, default=1, help="number of nodes")
```

You can then parse the arguments using the `parse_args` method.

```python
args = parser.parse_args()
```

You can access the arguments using the dot notation.

```python
print(args.nodes)
```


`os` module provides a way to interact with the operating system. We will use it to check if the file exists before reading it.

```python
import os
if os.path.exists(args.command):
    with open(args.command) as f:
        command = f.readlines()
else:
    command = args.command.split("\n")
```

Similar to `os.path.exists`, the `os.path.isfile` function checks if the file exists and `os.path.getsize` checks if the file is empty.


Now let's put this all together.


In [ ]:
import argparse
import os

def read_command_from_file(command_file):
    with open(command_file, 'r') as file:
        return file.read().strip()

def generate_slurm_script(args):
    with open(args.output_file, "w") as f:
        f.write(f"""#!/bin/bash
#SBATCH --nodes={args.nodes}
#SBATCH --ntasks-per-node={args.ntasks_per_node}
#SBATCH --partition={args.partition}
#SBATCH --mem={args.mem}
#SBATCH --time={args.time}
#SBATCH --account={args.account}
#SBATCH --job-name={args.job_name}
#SBATCH --output={args.output}
#SBATCH --error={args.error}
#SBATCH --mail-user={args.mail_user}
#SBATCH --mail-type={args.mail_type}

module load {args.module}
{args.command}
""")

def main():
    parser = argparse.ArgumentParser(description="Generate Slurm script for job submission")
    parser.add_argument("--nodes", type=int, default=1, help="Number of nodes")
    parser.add_argument("--ntasks-per-node", type=int, default=4, help="Number of tasks per node")
    parser.add_argument("--partition", default="nova", help="Partition name")
    parser.add_argument("--mem", default="24GB", help="Memory per node")
    parser.add_argument("--time", default="24:00:00", help="Walltime limit (HH:MM:SS or DD-HH:MM:SS)")
    parser.add_argument("--account", default="mhufford-lab", help="Account name")
    parser.add_argument("--job-name", default="my-cool-job", help="Job name")
    parser.add_argument("--output", default="sbatch_stdout.txt", help="Output file")
    parser.add_argument("--error", default="sbatch_stderr.txt", help="Error file")
    parser.add_argument("--mail-user", default="username@domain.com", help="Email address")
    parser.add_argument("--mail-type", default="begin", help="Email type (begin, end, fail)")
    parser.add_argument("--module", default="samtools", help="Module name")
    parser.add_argument("--output-file", help="Filename for the generated Slurm script")
    parser.add_argument("--command-file", help="Filename containing the command to run")
    args = parser.parse_args()

    if args.command_file:
        if os.path.isfile(args.command_file) and os.path.getsize(args.command_file) > 0:
            # Read command from file
            args.command = read_command_from_file(args.command_file)
        else:
            print("Error: Command file is empty or does not exist.")
            return
    else:
        print("Error: Please provide a command file.")
        return

    if args.output_file is None:
        # Generate output file name based on input command filename with job.sh suffix
        input_file_name = os.path.basename(args.command_file).replace('.txt', '')
        args.output_file = f"{input_file_name}_job.sh"

    generate_slurm_script(args)

if __name__ == "__main__": # this line is required to run the script from the command line
    main()

There are 3 main parts to this script:

1. `read_command_from_file` function reads the command from the file.
2. `generate_slurm_script` function generates the Slurm script.
3. `main` function parses the command-line arguments and generates the Slurm script.

You can save this script as `advanced_slurm_script.py` and you will have a more advanced script generator that can handle multi-line commands and generate the Slurm script for you.

Examples:

```bash
python advanced_slurm_script.py  --help
```

The output:
```
usage: myjob.py [-h] [--nodes NODES] [--ntasks-per-node NTASKS_PER_NODE] 
[--partition PARTITION] [--mem MEM] [--time TIME] [--account ACCOUNT] 
[--job-name JOB_NAME] [--output OUTPUT] [--error ERROR] [--mail-user MAIL_USER]
[--mail-type MAIL_TYPE] [--module MODULE] [--output-file OUTPUT_FILE] 
[--command-file COMMAND_FILE]

Generate Slurm script for job submission

options:
  -h, --help            show this help message and exit
  --nodes NODES         Number of nodes
  --ntasks-per-node NTASKS_PER_NODE
                        Number of tasks per node
  --partition PARTITION
                        Partition name
  --mem MEM             Memory per node
  --time TIME           Walltime limit (HH:MM:SS or DD-HH:MM:SS)
  --account ACCOUNT     Account name
  --job-name JOB_NAME   Job name
  --output OUTPUT       Output file
  --error ERROR         Error file
  --mail-user MAIL_USER
                        Email address
  --mail-type MAIL_TYPE
                        Email type (begin, end, fail)
  --module MODULE       Module name
  --output-file OUTPUT_FILE
                        Filename for the generated Slurm script
  --command-file COMMAND_FILE
                        Filename containing the command to run
```

Usage example:

My commands in the file `gzip.txt` contains following lines:

```bash
gzip Zm-B73-REFERENCE-NAM-5.0_Zm00001eb.1.gff3
gzip Zm-B97-REFERENCE-NAM-1.0_Zm00018ab.1.gff3
gzip Zm-CML322-REFERENCE-NAM-1.0_Zm00025ab.1.gff3
gzip Zm-CML333-REFERENCE-NAM-1.0_Zm00026ab.1.gff3
gzip Zm-CML52-REFERENCE-NAM-1.0_Zm00019ab.1.gff3
gzip Zm-HP301-REFERENCE-NAM-1.0_Zm00027ab.1.gff3
gzip Zm-M37W-REFERENCE-NAM-1.0_Zm00032ab.1.gff3
gzip Zm-NC350-REFERENCE-NAM-1.0_Zm00036ab.1.gff3
gzip Zm-Oh43-REFERENCE-NAM-1.0_Zm00039ab.1.gff3
gzip Zm-P39-REFERENCE-NAM-1.0_Zm00040ab.1.gff3
gzip Zm-Tzi8-REFERENCE-NAM-1.0_Zm00042ab.1.gff3
```

To generate the Slurm script for this command, you can run the following command:

```bash
python advanced_slurm_script.py --command-file gzip.txt
```

This will generate a Slurm script named `gzip_job.sh` with the following content:

```bash
#!/bin/bash
#SBATCH --nodes=1
#SBATCH --ntasks-per-node=4
#SBATCH --partition=nova
#SBATCH --mem=24GB
#SBATCH --time=24:00:00
#SBATCH --account=mhufford-lab
#SBATCH --job-name=my-cool-job
#SBATCH --output=sbatch_stdout.txt
#SBATCH --error=sbatch_stderr.txt
#SBATCH --mail-user=username@domain.com
#SBATCH --mail-type=begin

module load samtools
gzip Zm-B73-REFERENCE-NAM-5.0_Zm00001eb.1.gff3
gzip Zm-B97-REFERENCE-NAM-1.0_Zm00018ab.1.gff3
gzip Zm-CML322-REFERENCE-NAM-1.0_Zm00025ab.1.gff3
gzip Zm-CML333-REFERENCE-NAM-1.0_Zm00026ab.1.gff3
gzip Zm-CML52-REFERENCE-NAM-1.0_Zm00019ab.1.gff3
gzip Zm-HP301-REFERENCE-NAM-1.0_Zm00027ab.1.gff3
gzip Zm-M37W-REFERENCE-NAM-1.0_Zm00032ab.1.gff3
gzip Zm-NC350-REFERENCE-NAM-1.0_Zm00036ab.1.gff3
gzip Zm-Oh43-REFERENCE-NAM-1.0_Zm00039ab.1.gff3
gzip Zm-P39-REFERENCE-NAM-1.0_Zm00040ab.1.gff3
gzip Zm-Tzi8-REFERENCE-NAM-1.0_Zm00042ab.1.gff3
```

I can further customize it for my needs and submit it to the Slurm scheduler.

eg:

```bash
python advanced_slurm_script.py --command-file gzip.txt --job-name gzip-job --output gzip_stdout.txt --error gzip_stderr.txt
```
